## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise

 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


**Before you start attempting this project it is highly recommended that you finish the practice project.**


## Setup


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 15.9 MB/s eta 0:00:00


### Importing Required Libraries



In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [4]:
from pyspark.sql import SparkSession

from pyspark.ml.feature import StringIndexer, StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator

from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel


### Task 2 - Create a spark session


In [5]:
#Create a SparkSession

spark = SparkSession.builder.appName("final project ML").getOrCreate()

### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2024-06-21 19:22:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.1s    

2024-06-21 19:22:39 (520 KB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



Load the dataset into the spark dataframe


In [7]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv("NASA_airfoil_noise_raw.csv", inferSchema=True, header=True)


### Task 4 - Print top 5 rows of the dataset


In [8]:
df.show(5)


+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



### Task 6 - Print the total number of rows in the dataset


In [9]:
#your code goes here
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [10]:
df = df.dropDuplicates()


### Task 8 - Print the total number of rows in the dataset


In [11]:
#your code goes here

rowcount2 = df.count()
print(rowcount2)


1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [12]:
df = df.dropna()


### Task 10 - Print the total number of rows in the dataset


In [13]:
#your code goes here

rowcount3 = df.count()
print(rowcount3)


1499


In [14]:
df.printSchema()

root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevel: double (nullable = true)



In [17]:
print("n. of partitions: ", df.rdd.getNumPartitions())

n. of partitions:  200


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"


In [15]:
# your code goes here

df = df.withColumnRenamed("SoundLevel","SoundLevelDecibels")


### Task 12 - Save the dataframe in parquet format, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [16]:

!mkdir NASA_airfoil_noise_cleaned.parquet

df.write.mode("overwrite").parquet("./NASA_airfoil_noise_cleaned.parquet")


#### Part 1 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [18]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [19]:
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")


### Task 2 - Print the total number of rows in the dataset


In [20]:
rowcount4 = df.count()
print(rowcount4)

1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [21]:
inputed_lines = ["Frequency","AngleOfAttack","ChordLength","FreeStreamVelocity","SuctionSideDisplacement"]
assembler = VectorAssembler(inputCols=inputed_lines, outputCol="features")


### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [22]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


### Task 5 - Define the Model creation pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"

**Note:You need to use the scaledfeatures retreived in the previous step(StandardScaler pipeline stage).**


In [23]:
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")


### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [24]:
pipeline = Pipeline(stages=[assembler, scaler, lr])


### Task 7 - Split the data


In [25]:
(trainingData, testingData) = df.randomSplit([0.7,0.3], seed=42)

### Task 8 - Fit the pipeline


In [26]:
# Fit the pipeline using the training data

pipelineModel = pipeline.fit(trainingData)


#### Part 2 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [27]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [28]:
# Make predictions on testing data

predictions = pipelineModel.transform(testingData)


### Task 2 - Print the MSE


In [30]:
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared Error (MSE) =", mse)


Mean Squared Error (MSE) = 26.57450359380139


### Task 3 - Print the MAE


In [31]:
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE) =", mae)


Mean Absolute Error (MAE) = 3.9923220160490525


### Task 4 - Print the R-Squared(R2)


In [33]:
evaluator = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared =", r2)


R Squared = 0.4785118915245046


#### Part 3 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [34]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  26.57
Mean Absolute Error =  3.99
R Squared =  0.48
Intercept =  132.57


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [35]:
# Save the pipeline model as "Final_Project"
!mkdir Final_Project

pipelineModel.write().overwrite().save("./Final_Project")


### Task 2 - Load the model from the path "Final_Project"


In [36]:
# Load the pipeline model you have created in the previous step
loadedPipelineModel = PipelineModel.load("./Final_Project")


### Task 3 - Make predictions using the loaded model on the testdata


In [37]:
# Use the loaded pipeline model and make predictions using testingData
predictions = loadedPipelineModel.transform(testingData)


### Task 4 - Show the predictions


In [39]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
predictions.select("prediction").show(5)


+------------------+
|        prediction|
+------------------+
| 121.1759019027927|
|122.47988450139373|
|127.61142882533788|
|129.16838160755117|
|122.09809876229559|
+------------------+
only showing top 5 rows



#### Part 4 - Evaluation




**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [40]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.8025
Coefficient for AngleOfAttack is -2.1485
Coefficient for ChordLength is -3.3966
Coefficient for FreeStreamVelocity is 1.4629
Coefficient for SuctionSideDisplacement is -2.1833


In [41]:
spark.stop()